Import the preprepared dataset, indexed by system_metadata_id, and renaming columns to replace the '.' with _ to avoid later errors.

In [59]:
import requests
from urllib import parse
from pprint import pprint
import pandas as pd
import config
import numpy as np
import os, sys

display.max_rows = None


if not os.path.exists('data/cleaned_opendoar_data.csv'):
    print('data/cleaned_opendoar_data.csv doesnt exist, please run international_opendoar.py to create it.')
    sys.exit()

input_data = 'data/cleaned_opendoar_data.csv'
df_all_data = pd.read_csv(input_data, header=0)
df_all_data.columns=df_all_data.columns.str.replace('.','_')
df_all_data.set_index('system_metadata_id', inplace=True)

display(df_all_data)

 

/tmp/ipykernel_48993/2904952750.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_all_data.columns=df_all_data.columns.str.replace('.','_')


,organisation_country,organisation_url,repository_metadata_software_name,repository_metadata_metadata_record_count,repository_metadata_type,repository_metadata_url,repository_metadata_full_text_record_count,repository_metadata_oai_url,system_metadata_uri,repository_metadata_software_name_other
system_metadata_id,,,,,,,,,,
2,ch,https://www.unisg.ch,eprints,43608.0,institutional,http://www.alexandria.unisg.ch/,0.0,https://www.alexandria.unisg.ch/cgi/oai2,https://v2.sherpa.ac.uk/id/repository/2,NaN
3,it,http://www.unibo.it,eprints,5058.0,institutional,http://amsacta.unibo.it,4471.0,http://amsacta.unibo.it/cgi/oai2,https://v2.sherpa.ac.uk/id/repository/3,NaN
5,gb,https://www.lincoln.ac.uk,eprints,18422.0,institutional,https://eprints.lincoln.ac.uk,NaN,https://eprints.lincoln.ac.uk/cgi/oai2,https://v2.sherpa.ac.uk/id/repository/5,NaN
7,fr,http://www.ifremer.fr,NaN,26734.0,institutional,http://archimer.ifremer.fr,NaN,http://www.ifremer.fr/docelec/oai/OAIHandler,https://v2.sherpa.ac.uk/id/repository/7,NaN
8,fr,https://www.ccsd.cnrs.fr,hal,1787.0,institutional,http://edutice.archives-ouvertes.fr/,0.0,https://api.archives-ouvertes.fr/oai/edutice,https://v2.sherpa.ac.uk/id/repository/8,NaN
...,...,...,...,...,...,...,...,...,...,...
10147,fr,https://www.utt.fr,hal,NaN,institutional,https://hal-utt.archives-ouvertes.fr,NaN,https://api.archives-ouvertes.fr/oai/UTT,https://v2.sherpa.ac.uk/id/repository/10147,NaN
10148,fr,http://www.chartes.psl.eu,contentdm,NaN,institutional,https://hal-enc.archives-ouvertes.fr,NaN,NaN,https://v2.sherpa.ac.uk/id/repository/10148,NaN
10149,cl,https://www.ucentral.cl/,other,NaN,institutional,https://ucdc.ent.sirsidynix.net/client/es_CL/r...,NaN,https://ucdc.ent.sirsidynix.net/discovery-sear...,https://v2.sherpa.ac.uk/id/repository/10149,Sirsi Corporation


### Types of RIS

Examine the count of different RIS types (e.g., Pure, Eprints, Fedora etc).

In [60]:
''' Table of count of RIS types'''
vals = df_all_data.groupby("repository_metadata_software_name").size().sort_values(ascending=False)
display(vals)


repository_metadata_software_name
dspace             2035
other               537
eprints             537
weko                530
digital_commons     291
islandora           151
opus                 87
contentdm            78
hal                  66
pure                 60
dlibra               55
fedora               50
greenstone           28
drupal               27
invenio              23
wildfire             22
diva_portal          19
earmas               14
digitool             14
vital                12
mycore               11
samvera              10
digibib              10
omeka                 9
fez                   9
xoonips               7
dspace_cris           7
cybertesis            6
open_repository       4
equella               3
scielo                1
dtype: int64

&NewLine;  

This contains a lot of 'other' values, but `repository_metadata_software_name` can be used to replace the 'other' value for something more specific.

In [61]:
display(df_all_data["repository_metadata_software_name"])

df_all_data.loc[df_all_data.repository_metadata_software_name == 'other', 'repository_metadata_software_name'] = df_all_data.repository_metadata_software_name_other


system_metadata_id
2          eprints
3          eprints
5          eprints
7              NaN
8              hal
           ...    
10147          hal
10148    contentdm
10149        other
10150       dspace
10151    islandora
Name: repository_metadata_software_name, Length: 4971, dtype: object

Some cells contain the same data, but in slightly different formats (e.g., 

In [62]:
df_all_data['repository_metadata_software_name'] = df_all_data['repository_metadata_software_name'].str.replace('_', ' ')
df_all_data['repository_metadata_software_name'] = df_all_data['repository_metadata_software_name'].str.replace('-', ' ')
display(df_all_data['repository_metadata_software_name'])

system_metadata_id
2                  eprints
3                  eprints
5                  eprints
7                      NaN
8                      hal
               ...        
10147                  hal
10148            contentdm
10149    Sirsi Corporation
10150               dspace
10151            islandora
Name: repository_metadata_software_name, Length: 4971, dtype: object

We can now redo the table with better detail, including ignoring case to better group values.

In [79]:
''' Table of count of RIS types'''
df_all_data["repository_metadata_software_name"] = df_all_data["repository_metadata_software_name"].str.lower()
vals = df_all_data.groupby("repository_metadata_software_name".lower()).size().sort_values(ascending=False)
display(vals)
vals.to_csv('./vals.csv')

repository_metadata_software_name
dspace             2035
eprints             537
weko                530
digital commons     435
islandora           151
                   ... 
imperia               1
indigo                1
infolib dbr           1
infolibクラウドサービス       1
独自                    1
Length: 199, dtype: int64

and dump to latex if needed.

In [64]:
print(vals.to_latex())

\begin{tabular}{lr}
\toprule
{} &     0 \\
repository\_metadata\_software\_name        &       \\
\midrule
dspace                                   &  2035 \\
eprints                                  &   537 \\
weko                                     &   530 \\
digital commons                          &   291 \\
islandora                                &   151 \\
Digital Commons                          &   144 \\
opus                                     &    87 \\
contentdm                                &    78 \\
hal                                      &    66 \\
pure                                     &    60 \\
dlibra                                   &    57 \\
fedora                                   &    50 \\
greenstone                               &    28 \\
drupal                                   &    27 \\
invenio                                  &    23 \\
wildfire                                 &    22 \\
diva portal                              &    19 \\
HTML     

/tmp/ipykernel_48993/2121909498.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(vals.to_latex())


In [65]:
vals.groupby('value')['tempx'].apply(' '.join).reset_index()

KeyError: 'value'

In [ ]:
df_all_data.to_csv('./all_data.csv')